# Multigrid Solvers for scalar unfitted interface problems

### PDE

We solve the following interface problem:
\begin{align*}
  - \mu_i \Delta u &= f_i \quad \text{ in } \Omega_i, \tag{P1}\\
  [\![ u ]\!] &= 0 \quad \text{ on } \Gamma, \tag{P2} \\
  [\![ - \mu \frac{\partial u}{\partial n} ]\!] &= 0 \quad \text{ on } \Gamma. \tag{P3}
\end{align*}
Here, $\Omega_1$ and $\Omega_2$ are two disjoint subdomains that fill the background domain that is assumed to be meshed. $\Gamma$ is the dividing surface between $\Omega_1$ and $\Omega_2$ and it is assumed that $\Gamma$ is not aligned. $\mu$ is a domain-wise constant coefficient which may be different in the two subdomains.

### Imports

In [ ]:
# solve the interface Poisson equation - mu Delta u = f in Omega_i
# with interface conditions                     [u] = 0 on Gamma
#                                       [-mu du/dn] = 0 on Gamma
# with Dirichlet boundary condition u = 0

from ngsolve.meshes import*
from ngsolve import *
from ngsolve.krylovspace import CG
from netgen.occ import *
from xfem import *
from xfem.cutmg import MultiGridCL, CutFemSmoother, LinearMGIterator
from xfem.lsetcurv import *

from ngsolve.webgui import *
ngsglobals.msg_level = 1

### Background mesh

In [ ]:
# generate a triangular mesh of mesh-size 0.2

def MakeCoarseMesh():
    square = OCCGeometry(unit_square_shape.Scale((0,0,0),3).Move((-1.5,-1.5,0)), dim=2)
    mesh = Mesh (square.GenerateMesh(maxh=0.2, quad_dominated=False))
    return mesh

mesh = MakeCoarseMesh()
Draw(mesh)

Here we choose the coefficients (you may play around with these parameters later):

In [ ]:
mu = [1e-5, 1]

### Example with analytical solution

The interfrace $\Gamma$ is defined by the zero level of the levelset function

In [ ]:
levelset = sqrt(x**2+y**2) - 1

Analytical solution and corresponding data functions are given by:

In [ ]:
rhsscal = -4*mu[0]*mu[1] 
coef_f = [ rhsscal, rhsscal ]
distsq = x**2 + y**2 - 1
solution = [ mu[1] * distsq, mu[0] * distsq ]

### Geometry through level set P1 representation:
the approximate interface is given by the linearized levelset function

In [ ]:
def GetLevelSetGeometry(mesh):
    lsetp1 = GridFunction(H1(mesh, order = 1))
    InterpolateToP1(levelset,lsetp1)
    ci = CutInfo(mesh, lsetp1)   
    return ci, lsetp1

### Discretization:
We use the cut finite element space

In [ ]:
def GetCutFESpace(mesh):
    Vh = H1(mesh, order=1, dirichlet=[1,2,3,4])
    VhNeg = Compress( Vh, active_dofs = GetDofsOfElements(Vh,ci.GetElementsOfType(HASNEG)))
    VhPos = Compress( Vh, active_dofs = GetDofsOfElements(Vh,ci.GetElementsOfType(HASPOS)))
    CutVh = FESpace( [VhNeg, VhPos], flags={"dgjumps": True} )
    return CutVh

with the coefficient stable discretization
$$
 A_h(u_h,v_h) = a_h(u_h,v_h) + \varepsilon_g j_h(u_h,v_h)
$$
$u_h, v_h \in V_h^- \times V_h^+$
and Poisson part
$$
  a_h(u_h,v_h) = \sum_{i=1,2} \int_{\Omega_i} \mu_i \nabla u_{i,h} \cdot \nabla v_{i,h} ~dx
  - \int_\Gamma \{\mu \nabla u_h \cdot n_\Gamma\}_\mu [v_h] ~ds 
  - \int_\Gamma \{\mu \nabla v_h \cdot n_\Gamma\}_\mu [u_h] ~ds \\
  + \frac{\lambda_N}{h} \frac{\mu_1\mu_2}{\mu_1 + \mu_2}\int_\Gamma [u_h][v_h] ~ds
$$
using the averaging 
$$\{u_h\}_\mu = \frac{\mu_2}{\mu_1+\mu_2} u_{1,h} + \frac{\mu_2}{\mu_1+\mu_2} u_{2,h}$$
and the ghost penalty term 
$$
 j_h(u_h,v_h) = \sum_{i=1,2} \sum_{F \in \mathcal{F_i}} \frac{\mu_i}{h^2} \int_{\omega_F}(u_{i,h}|_{T_1} - u_{i,h}|_{T_2}) (v_{i,h}|_{T_1} - v_{i,h}|_{T_2}) ~dx
$$

![alt](graphics/macro-element.png) 

with the discretization parameters:

In [ ]:
gamma_stab = 10     # Ghost penalty
lambda_nitsche = 10 # Nitsche penalty
nref = 3            # number of refinements

### Assembling of CutFEM matrix:

In [ ]:
def AssembleCutPoisson(**kwargs):
    ci = kwargs['ci']
    lsetp1 = kwargs['lsetp1']
    order = kwargs['order']
    CutVh = kwargs['CutVh']
    
    
    # face sets for ghost penalty
    ba_facets   = [GetFacetsWithNeighborTypes(mesh,
                                              a=ci.GetElementsOfType(HASNEG),
                                              b=ci.GetElementsOfType(IF)),
                   GetFacetsWithNeighborTypes(mesh,
                                              a=ci.GetElementsOfType(HASPOS),
                                              b=ci.GetElementsOfType(IF))
                  ]

    n_lset  = 1.0/grad(lsetp1).Norm() * grad(lsetp1)
    h       = specialcf.mesh_size

    kappa = [ mu[1] / ( mu[0] + mu[1] ) , mu[0] /  ( mu[0] + mu[1] ) ]
    
    # stabilization parameter for Nitsche
    nitsche_penalty  = 2*mu[0]*mu[1]/(mu[0] + mu[1]) * lambda_nitsche * order * order        
        
    # expressions of test and trial functions (u and v are tuples):
    (u,v) = CutVh.TnT()

    gradu = [grad(ui) for ui in u]
    gradv = [grad(vi) for vi in v]

    average_flux_u = sum([- kappa[i] * mu[i] * gradu[i] * n_lset for i in [0,1]])
    average_flux_v = sum([- kappa[i] * mu[i] * gradv[i] * n_lset for i in [0,1]])
    jmp_u = u[0] - u[1]
    jmp_v = v[0] - v[1]

    # integration domains
    lset_doms = [{ "levelset" : lsetp1, "domain_type" : dt} for dt in [NEG,POS,IF]]
    # bilinear forms:
    a = BilinearForm(CutVh, symmetric = False)


    #domain integrals:
    for i in [0,1] :
        a += SymbolicBFI( lset_doms[i], form = mu[i] * gradu[i] * gradv[i] )        

    # Nitsche integrals:
    a += SymbolicBFI(lset_doms[2], form = average_flux_u * jmp_v
                                        + average_flux_v * jmp_u
                                        + nitsche_penalty / h * jmp_u * jmp_v ) 

    for i in [0,1]:
        # Ghost penalty
        a += SymbolicFacetPatchBFI( form =  gamma_stab  * 
                                    mu[i] / h / h * 
                                    (u[i] - u[i].Other()) * (v[i] - v[i].Other()), 
                                    skeleton=False, definedonelements=ba_facets[i] )


    #if( order > 1) and deformation:
    #    mesh.SetDeformation( deformation )

    # setting up matrix and vector
    a.Assemble()    
    
    f = LinearForm(CutVh)        
    for i in [0,1] :
        f += SymbolicLFI( lset_doms[i], form = coef_f[i] * v[i] )
    f.Assemble()
    
    return (a,f)

### Measuring the error

In [ ]:
def ComputeL2Error(gfu,lsetp1):    # Computation of L2 error:
    err_sqr_coefs = [(gfu.components[i]-solution[i])*(gfu.components[i]-solution[i]) for i in [0,1] ]
    lset_doms = [{ "levelset" : lsetp1, "domain_type" : dt} for dt in [NEG,POS]]
    l2error = sqrt(sum([Integrate(lset_doms[i], cf=err_sqr_coefs[i], mesh=mesh, order=2*1) for i in [0,1] ]))   
    return l2error

In [ ]:
def CreateLsetMeshAdapt(order):
    lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=1000, discontinuous_qn=True)
    deformation = lsetmeshadap.CalcDeformation(levelset)
    lsetp1 = lsetmeshadap.lset_p1

    return (lsetmeshadap, deformation, lsetp1 )


In [ ]:
Vh = H1(mesh, order=1, dirichlet=[1,2,3,4])

In [ ]:
(lsetmeshadap, deformation, lsetp1 ) = CreateLsetMeshAdapt(order=1)

In [ ]:
ci = CutInfo(mesh,lsetp1)

### Solution with a direct solver

In [ ]:
for i in range(nref):
    mesh.Refine()

In [ ]:
lsetmeshadap, deformation, lsetp1 = CreateLsetMeshAdapt(order=1)
ci = CutInfo(mesh,lsetp1)
CutFes = GetCutFESpace(mesh)

In [ ]:

a, f = AssembleCutPoisson( CutVh=CutFes, ci=ci, lsetp1=lsetp1, order=1 )

#updating with non-homogeneous boundary conditions
gfu = GridFunction( CutFes )
#boundary lies in positive part
gfu.components[1].Set( solution[1], BND )
#account for boundary conditions
f.vec.data -= a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(CutFes.FreeDofs()) * f.vec
l2error = ComputeL2Error(gfu,lsetp1)
print("L2 error : ",l2error)

# Solution of linear systems with a (Cut)MultiGrid

We re-initialize the mesh

In [ ]:
mesh = MakeCoarseMesh()

#### A smoother tailored for interface CutFEM problems:
The smoother essentially implements the $\texttt{Smooth}$ operation and is loaded via the xfem.cutmg library.
It consists of standard (Gauss-Seidel) smoothing in the subdomains and is followed by an interface correction on the cut elements

\begin{align*}
  x &= S(x,b) \quad {\small\text{#standard Gauss Seidel smoothing}}  \\
  x &= x - (R^\Gamma)^T (A^\Gamma)^{-1} R^\Gamma ( Ax -b ) \quad {\small\text{#interface smoothing}}
\end{align*}

such that $A^\Gamma$ is the restriction of $A$ to the interface band $\Omega_\Gamma$ and $R^\Gamma$ the corresponding mapping

<center>![alt](graphics/if-patch.png)</center>

We can choose between running the multigrid method as a preconditioner inside a CG solver or within a Richardson iteration

In [ ]:
# run multigrid as preconditioner inside a CG method
# or as a standalone (Richardson) iteration
MGprecond = False

if( MGprecond ):
    mgmaxit = 1      #only 1 V-cycle as preconditioning
    mgprint = False  #don't print anything when being used as a preconditioner
else:
    mgmaxit = 30
    mgprint = True

### Multigrid solver for P1 discretization
We refine the mesh, solve the system by the multigrid solver and compute the current error.

For the coarse level matrices we use direct discretization.

In [ ]:
for i in range(nref+1):
    if i == 0:
        ci, lsetp1 = GetLevelSetGeometry(mesh)        
        CutVh = GetCutFESpace(mesh)
        gfu = GridFunction(CutVh)
        #ucoef = IfPos(  )
        scene = DrawDC(lsetp1, gfu.components[1], gfu.components[0], mesh, 'u') 
    else:
        #mesh update
        mesh.Refine()
        
        #lset geom update
        lsetp1.space.Update()
        lsetp1.Update()    
        InterpolateToP1(levelset,lsetp1)
        ci = CutInfo(mesh,lsetp1)        
        
        #cut space update
        Vh = CutVh.components[0].GetBaseSpace()
        Vh.Update()
        CutVh.components[0].SetActiveDofs( GetDofsOfElements(Vh,ci.GetElementsOfType(HASNEG)))
        CutVh.components[1].SetActiveDofs( GetDofsOfElements(Vh,ci.GetElementsOfType(HASPOS)))
        CutVh.Update()

        gfu.Update()
        
    a,f = AssembleCutPoisson( CutVh=CutVh, ci=ci, lsetp1=lsetp1, order=1 )

    if i == 0:
        # initialize mg iterator (for p1 elements)
        # we use nu=1 smoothing step for pre- and post-smoothing
        # and pass the "AssembleCutPoisson" function for setting up the coarse grid matrices
        mgiter = LinearMGIterator(a=a, ci=ci, lsetp1=lsetp1, nref=nref, mesh=mesh,
                                  nu=1,maxit=mgmaxit, printinfo=mgprint, ProlType=P1Prolongation,
                                  ifsolver="cg", assemblefct=AssembleCutPoisson )
    else:
        mgiter.Update(a,CutVh,ci)
        
    gfu.components[1].Set( solution[1], BND )
    f.vec.data -= a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    
    if( MGprecond ):
        # watch out, when comparing the number of iterations
        # here the tolerance is prescribed for ||e||_A instead of ||r||_2
        update = CG(a.mat, f.vec, pre=mgiter, maxsteps=20, tol=1e-6,printrates=True)
    else:
        update = mgiter * f.vec

    gfu.vec.data += update
    
    # Computation of L2 error:
    err_sqr_coefs = [(gfu.components[i]-solution[i])*(gfu.components[i]-solution[i]) for i in [0,1] ]
    lset_doms = [{ "levelset" : lsetp1, "domain_type" : dt} for dt in [NEG,POS,IF]]    
    l2error = sqrt( sum( [Integrate(lset_doms[i], cf=err_sqr_coefs[i], mesh=mesh, order=2*1, heapsize=1000000) for i in [0,1] ]))   
    print("L2 error : ",l2error) 
    
    scene.Redraw()    